In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.list_physical_devices(device_type=None)

import keras
from keras import layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import regularizers

In [ ]:
#load data
train_label=np.load('train_label.npy')
label=train_label[:59976].astype(int)
print(label.shape)
filtered=np.load('filtered_49-1-weak.npy')
filtered=filtered[:,:,:,1:]
print(filtered.shape)

In [3]:
#digital with Convolutional layer

l2r=0
l1r=0
br=0
model=keras.Sequential()
model.add(layers.Reshape((28,28,1),input_shape=(28,28)))
model.add(layers.Conv2D(kernel_size=3,filters=8))
model.add(layers.Activation(tf.square))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(500,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=l1r, l2=l2r),
    bias_regularizer=regularizers.l2(br),
    activity_regularizer=regularizers.l2(1e-5)))
model.add(layers.Dense(10,kernel_regularizer=regularizers.l1_l2(l1=l1r, l2=l2r),
    bias_regularizer=regularizers.l2(br),
    activity_regularizer=regularizers.l2(1e-5)))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 8)         80        
_________________________________________________________________
activation (Activation)      (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 8)         32        
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               6

In [ ]:
for i in range(1):
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.RMSprop(),
        metrics=["accuracy"],
    )

    earlystopping = keras.callbacks.EarlyStopping(monitor ="val_loss",  
                                                mode ="min", patience = 25,  
                                                restore_best_weights = True,
                                               verbose=1) 

    cutoff=50000

    history = model.fit(filtered[:cutoff], label[:cutoff], batch_size=128, epochs=1000, callbacks=[earlystopping], validation_data=(filtered[cutoff:], label[cutoff:]),verbose=1)

    test_scores = model.evaluate(filtered[cutoff:], label[cutoff:], verbose=2)
    print("Test loss:", test_scores[0])
    print("Test accuracy:", test_scores[1])

In [4]:
#optical hybrid
l2r=0
l1r=0
br=0


model=keras.Sequential()
model.add(layers.MaxPooling2D(pool_size=(2,2),input_shape=(30,30,8)))
model.add(layers.BatchNormalization())
#model.add(layers.LayerNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(500,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=l1r, l2=l2r),
    bias_regularizer=regularizers.l2(br),
    activity_regularizer=regularizers.l2(1e-5)))
model.add(layers.Dense(10,kernel_regularizer=regularizers.l1_l2(l1=l1r, l2=l2r),
    bias_regularizer=regularizers.l2(br),
    activity_regularizer=regularizers.l2(1e-5)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 8)         32        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               900500    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
Total params: 905,542
Trainable params: 905,526
Non-trainable params: 16
_________________________________________________________________


In [ ]:
for i in range(1):
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.RMSprop(),
        metrics=["accuracy"],
    )

    earlystopping = keras.callbacks.EarlyStopping(monitor ="val_loss",  
                                                mode ="min", patience = 25,  
                                                restore_best_weights = True,
                                               verbose=1) 

    cutoff=50000

    history = model.fit(filtered[:cutoff], label[:cutoff], batch_size=128, epochs=1000, callbacks=[earlystopping], validation_data=(filtered[cutoff:], label[cutoff:]),verbose=1)

    test_scores = model.evaluate(filtered[cutoff:], label[cutoff:], verbose=2)
    print("Test loss:", test_scores[0])
    print("Test accuracy:", test_scores[1])